In [ ]:
# расскоментируйте код ниже, чтобы установить все зависимости
# !pip install -q \
#     pyarrow==12.0.1 \
#     polars==0.18.6 \
#     pandas==2.0.3 \
#     optuna==3.3.0 \
#     tqdm==4.65.0 \
#     numpy==1.24.3 \
#     redis==4.6.0 \
#     gensim==4.3.2

In [ ]:
# раскоментируйте код ниже, чтобы скачать данные
# !wget -q https://files.grouplens.org/datasets/movielens/ml-100k.zip
# !unzip -q ml-100k.zip

In [1]:
import uuid
import redis
import optuna
import random

import polars as pl
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models import Word2Vec

from typing import List, Any

/home/i.nurgaliev/recsys/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t',
    names=['user_id', 'item_id', 'rating', 'timestamp']
)
ratings = pl.from_pandas(ratings).filter(pl.col('rating') >= 4)
ratings

user_id,item_id,rating,timestamp
i64,i64,i64,i64
298,474,4,884182806
253,465,5,891628467
286,1014,5,879781125
200,222,5,876042340
122,387,5,879270459
291,1042,4,874834944
119,392,4,886176814
167,486,4,892738452
299,144,4,877881320


In [3]:
grouped_df = (
    ratings
    .groupby('user_id')
    .agg([
        pl.col('item_id').apply(lambda x: x[:-3]).alias('train_ids'),
        pl.col('rating').apply(lambda x: x[:-3]).alias('train_ratings'),
        pl.col('item_id').apply(lambda x: x[-3:]).alias('test_ids'),
        pl.col('rating').apply(lambda x: x[-3:]).alias('test_ratings'),
    ])
)
grouped_df

user_id,train_ids,train_ratings,test_ids,test_ratings
i64,list[i64],list[i64],list[i64],list[i64]
192,"[1061, 1160, … 100]","[4, 4, … 5]","[289, 9, 287]","[4, 5, 4]"
336,"[1047, 1057, … 1118]","[4, 4, … 4]","[273, 50, 173]","[5, 4, 5]"
160,"[234, 174, … 250]","[5, 5, … 4]","[488, 952, 127]","[5, 4, 5]"
256,"[452, 781, … 21]","[4, 5, … 4]","[829, 597, 230]","[4, 4, 4]"
928,"[8, 98, … 168]","[5, 5, … 5]","[191, 135, 9]","[5, 4, 5]"
384,"[272, 355, … 327]","[5, 4, … 4]","[879, 258, 286]","[4, 4, 4]"
32,"[1012, 249, … 7]","[4, 4, … 4]","[628, 50, 181]","[4, 4, 4]"
528,"[239, 58, … 202]","[5, 5, … 5]","[79, 213, 410]","[5, 4, 4]"
64,"[381, 736, … 732]","[4, 4, … 4]","[651, 503, 184]","[4, 4, 4]"


In [4]:
TOP_K = 10


def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))


def user_hitrate(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: 1 if top-k recommendations contains at lease one relevant item
    """
    return int(user_intersection(y_rel, y_rec, k) > 0)


def user_precision(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / k


def user_ap(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: average precision metric for user recommendations
    """
    return np.sum([
        user_precision(y_rel, y_rec, idx + 1)
        for idx, item in enumerate(y_rec[:k]) if item in y_rel
    ]) / k

## Применим алгоритм w2v

Внутри gensim написан эффективный метод `predict_output_word`, позволяющий получать следующие возможные токены (в нашем случае объекты для рекомендации), перед фильтрацией просмотренного попросим его сгенерировать на `len(train_ids)` токенов больше (в худшем случае, нам придется все отфильтровать)

In [5]:
def evaluate_model(model):
    ap_list = []
    hitrate_list = []
    for train_ids, y_rel in grouped_df.select('train_ids', 'test_ids').rows():
        model_preds = model.predict_output_word(
            train_ids, topn=(TOP_K + len(train_ids))
        )
        if model_preds is None:
            hitrate_list.append(0)
            continue

        y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
        ap_list.append(user_ap(y_rel, y_rec))
        hitrate_list.append(user_hitrate(y_rel, y_rec))
    return np.mean(ap_list), np.mean(hitrate_list)

# обучим w2v с параметрами по умолчанию
model = Word2Vec(grouped_df['train_ids'].to_list())
mean_ap, mean_hitrate = evaluate_model(model)
print(f'MAP@{TOP_K} = {mean_ap:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

MAP@10 = 0.0026 Hitrate@10 = 0.1083


## Подберем самые оптимальные гиперпараметры с помощью optuna

Дла алгоритма [W2V](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec) рассмотрим следующие параметры:

- `sg` – 1, если использовать skip-gram, иначе cbow
- `window` – размер окна для обучения алгоритма w2v
- `ns_exponent` – степень популярности объектов, которая будет использована для negative sampling
- `negative` – количество негативных примеров для сэмплирования
- `min_count` – минимальное число взаимодействий, нужное для 
- `vector_size` – размерность эмбеддингов

In [6]:
SEED = 42

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

def objective(trial):
    sg = trial.suggest_categorical('sg', [0, 1])
    window = trial.suggest_int('window', 1, 10)
    ns_exponent = trial.suggest_float('ns_exponent', -3, 3)
    negative = trial.suggest_int('negative', 3, 20)
    min_count = trial.suggest_int('min_count', 0, 20)
    vector_size = trial.suggest_categorical('vector_size', [16, 32, 64, 128])
    
    print({
        'sg': sg,
        'window_len': window,
        'ns_exponent': ns_exponent,
        'negative': negative,
        'min_count': min_count,
        'vector_size': vector_size,
    })
    
    set_seed(SEED)
    model = Word2Vec(
        grouped_df['train_ids'].to_list(),
        window=window,
        sg=sg,
        hs=0,
        min_count=min_count,
        vector_size=vector_size,
        negative=negative,
        ns_exponent=ns_exponent,
        seed=SEED,
        epochs=10,
    )
    
    mean_ap, mean_hitrate = evaluate_model(model)
    print(f'MAP@{TOP_K} = {mean_ap:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')
    return mean_ap
    
    
study = optuna.create_study(directions=('maximize',))
study.optimize(objective, n_trials=100)

study.best_params

[I 2023-10-21 10:33:00,127] A new study created in memory with name: no-name-62dfb982-8983-4bce-9c11-da16b4298e3c


{'sg': 1, 'window_len': 4, 'ns_exponent': 2.7581226625078106, 'negative': 5, 'min_count': 3, 'vector_size': 32}


[I 2023-10-21 10:33:01,143] Trial 0 finished with value: 0.004109464707868963 and parameters: {'sg': 1, 'window': 4, 'ns_exponent': 2.7581226625078106, 'negative': 5, 'min_count': 3, 'vector_size': 32}. Best is trial 0 with value: 0.004109464707868963.


MAP@10 = 0.0041 Hitrate@10 = 0.1062
{'sg': 0, 'window_len': 2, 'ns_exponent': -1.3562907906098454, 'negative': 15, 'min_count': 5, 'vector_size': 64}


[I 2023-10-21 10:33:01,898] Trial 1 finished with value: 0.00873032759202972 and parameters: {'sg': 0, 'window': 2, 'ns_exponent': -1.3562907906098454, 'negative': 15, 'min_count': 5, 'vector_size': 64}. Best is trial 1 with value: 0.00873032759202972.


...

[I 2023-10-21 10:37:21,885] Trial 98 finished with value: 0.028558130699088146 and parameters: {'sg': 1, 'window': 8, 'ns_exponent': 0.15214988812821462, 'negative': 7, 'min_count': 18, 'vector_size': 32}. Best is trial 91 with value: 0.034732480580884836.


MAP@10 = 0.0286 Hitrate@10 = 0.4915
{'sg': 1, 'window_len': 6, 'ns_exponent': 0.3833942257883662, 'negative': 12, 'min_count': 2, 'vector_size': 32}


[I 2023-10-21 10:37:25,726] Trial 99 finished with value: 0.012993287740628167 and parameters: {'sg': 1, 'window': 6, 'ns_exponent': 0.3833942257883662, 'negative': 12, 'min_count': 2, 'vector_size': 32}. Best is trial 91 with value: 0.034732480580884836.


MAP@10 = 0.0130 Hitrate@10 = 0.3217


{'sg': 1,
 'window': 7,
 'ns_exponent': -0.020629396237178063,
 'negative': 9,
 'min_count': 17,
 'vector_size': 128}

In [7]:
set_seed(SEED)
model = Word2Vec(
    grouped_df['train_ids'].to_list(),
    **study.best_params,
    hs=0,
    seed=SEED,
    epochs=50
)

hitrate_list = []
for train_ids, y_rel in grouped_df.select('train_ids', 'test_ids').rows():
    model_preds = model.predict_output_word(train_ids, topn=(TOP_K + len(train_ids)))
    if model_preds is None:
        hitrate_list.append(0)
        continue
        
    y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
    hitrate_list.append(user_hitrate(y_rel, y_rec))

mean_ap, mean_hitrate = evaluate_model(model)
print(f'MAP@{TOP_K} = {mean_ap:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

MAP@10 = 0.0353 Hitrate@10 = 0.5499


## Сохраним рекомендации в redis

В реальных системах в качестве идентификаторов скорее всего будет использоваться uuid, тогда как при обучении удобно использовать представление в виде целых чисел.

Одним из вариантов работы с идентификаторами является создание словарей `user_ids_mapping` и `user_ids_inverse_mapping`, где первый делает преобразование _uuid -> int_, а второй _int -> uuid_.

Давайте просимулируем реальную рекомендательную систему и загрузим рекомендации в **redis**.

In [8]:
user_ids_inverse_mapping = {k: uuid.uuid4() for k in ratings['user_id'].unique()}
item_ids_inverse_mapping = {k: uuid.uuid4() for k in ratings['item_id'].unique()}

In [9]:
# если redis запущен в том же окружении, то можно использовать localhost
# иначе, измените host на ip-адресс сервера с запущенным redis
r = redis.Redis(host='localhost', db=0)
used_memory_before = r.info('memory')['used_memory']

In [10]:
TOP_K = 100  # сохраним топ-100 рекомендаций

for user_id, train_ids in grouped_df.select('user_id', 'train_ids').rows():
    model_preds = model.predict_output_word(train_ids, topn=(TOP_K + len(train_ids)))
    if model_preds is None:
        continue
        
    y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
    r.rpush(
        str(user_ids_inverse_mapping[user_id]),
        *[str(item_ids_inverse_mapping[item_id]) for item_id in y_rec]
    )

In [11]:
# прочитаем сохраненные рекомендации по ключу user_id
r.lrange(str(user_ids_inverse_mapping[1]), 0, TOP_K)

[b'64a78b2d-98b5-4734-a5a3-4580df5b298e',
 b'ae49927d-850e-43b2-96ef-6dfcfecb97b0',
 b'd50a1c43-1a0b-4c25-a65a-db58f3ebdf89',
 b'17189576-3284-492d-9e72-a5bda874c43a',
 b'92157072-1cd6-495d-a079-3822cbfbd26c',
 b'af769df0-4b8b-4afe-b878-82cacd978b12',
 b'e5cc28ba-b163-435a-9f84-85a5e9c1fe48',
 b'2d9ed025-4e10-4972-a43b-b7c706c0b0eb',
 b'a50a97b9-b9f2-45a0-b6d6-d3594a433a05',
 b'bf74a277-17fc-4a03-bb4c-c8d55086ad5b',
 b'd8b04319-7cfc-4492-acc8-b41f71a7d29b',
 b'bda01853-6a53-4420-8f17-a6584f4a60a0',
 b'cb2d057a-08e8-4580-a105-b5780a9aef69',
 b'bb18e13f-4f46-4a1c-8fc3-be8b2beef2b2',
 b'ab431b53-5fce-40a8-b41a-6de6e7b426af',
 b'22fb29ae-6b4e-4daf-9bf9-166210eced0f',
 b'a2eabe72-2922-4cb4-a2c3-2e88d586ae44',
 b'9f49f742-7a96-4d59-8f46-b56bc94bad73',
 b'5d53687c-1704-4a3e-a4d6-ef5016dd1871',
 b'9f34c9cc-515e-4c19-935a-dfee72e5749e',
 b'cb59cf96-f56e-4091-b3ce-b8333cdcc523',
 b'4da5b153-b211-4902-b06f-7497cd94ca09',
 b'2f287950-266c-494b-a2ee-da14775cebeb',
 b'd7ca1748-8c16-40c5-945d-6639787

In [12]:
r.info('memory')['used_memory'] - used_memory_before

4764320

Потратили около 5Мб на сохранение рекомендаций

In [13]:
r = redis.Redis(db=1)
used_memory_before = r.info('memory')['used_memory']

In [14]:
TOP_K = 100  # сохраним топ-100 рекомендаций

for user_id, train_ids in grouped_df.select('user_id', 'train_ids').rows():
    model_preds = model.predict_output_word(train_ids, topn=(TOP_K + len(train_ids)))
    if model_preds is None:
        continue
        
    y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
    r.rpush(
        user_ids_inverse_mapping[user_id].bytes,
        *[item_ids_inverse_mapping[item_id].bytes for item_id in y_rec]
    )

In [15]:
# прочитаем сохраненные рекомендации по ключу user_id
# для этого сначала преобразуем user_uuid в байты, а затем байты в item_uuid
[
    uuid.UUID(bytes=item_id_bytes, version=4)
    for item_id_bytes in r.lrange(user_ids_inverse_mapping[1].bytes, 0, TOP_K)
]

[UUID('64a78b2d-98b5-4734-a5a3-4580df5b298e'),
 UUID('ae49927d-850e-43b2-96ef-6dfcfecb97b0'),
 UUID('d50a1c43-1a0b-4c25-a65a-db58f3ebdf89'),
 UUID('17189576-3284-492d-9e72-a5bda874c43a'),
 UUID('92157072-1cd6-495d-a079-3822cbfbd26c'),
 UUID('af769df0-4b8b-4afe-b878-82cacd978b12'),
 UUID('e5cc28ba-b163-435a-9f84-85a5e9c1fe48'),
 UUID('2d9ed025-4e10-4972-a43b-b7c706c0b0eb'),
 UUID('a50a97b9-b9f2-45a0-b6d6-d3594a433a05'),
 UUID('bf74a277-17fc-4a03-bb4c-c8d55086ad5b'),
 UUID('d8b04319-7cfc-4492-acc8-b41f71a7d29b'),
 UUID('bda01853-6a53-4420-8f17-a6584f4a60a0'),
 UUID('cb2d057a-08e8-4580-a105-b5780a9aef69'),
 UUID('bb18e13f-4f46-4a1c-8fc3-be8b2beef2b2'),
 UUID('ab431b53-5fce-40a8-b41a-6de6e7b426af'),
 UUID('22fb29ae-6b4e-4daf-9bf9-166210eced0f'),
 UUID('a2eabe72-2922-4cb4-a2c3-2e88d586ae44'),
 UUID('9f49f742-7a96-4d59-8f46-b56bc94bad73'),
 UUID('5d53687c-1704-4a3e-a4d6-ef5016dd1871'),
 UUID('9f34c9cc-515e-4c19-935a-dfee72e5749e'),
 UUID('cb59cf96-f56e-4091-b3ce-b8333cdcc523'),
 UUID('4da5b1

In [16]:
r.info('memory')['used_memory'] - used_memory_before

2277120

Потратили в 2 раза меньше памяти для сохранения рекомендаций 🎉🎉